# Import modules

In [151]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import sin, cos, sqrt, atan2, radians

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'test', 'sample_submission', 'train', 'train.csv', 'sample_submission.csv']


# Load datas train

In [152]:
df_train = pd.read_csv('../input/train.csv', index_col='id', parse_dates=['pickup_datetime', 'dropoff_datetime'])
df_train.head(10)

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
id,,,,,,,,,,
id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435
id0801584,2,2016-01-30 22:01:40,2016-01-30 22:09:03,6,-73.982857,40.742195,-73.992081,40.749184,N,443
id1813257,1,2016-06-17 22:34:59,2016-06-17 22:40:40,4,-73.969017,40.757839,-73.957405,40.765896,N,341
id1324603,2,2016-05-21 07:54:58,2016-05-21 08:20:49,1,-73.969276,40.797779,-73.922470,40.760559,N,1551
id1301050,1,2016-05-27 23:12:23,2016-05-27 23:16:38,1,-73.999481,40.738400,-73.985786,40.732815,N,255


In [153]:
def create_datetime(df):
    df['pickup_year'] = df['pickup_datetime'].dt.year
    df['pickup_month'] = df['pickup_datetime'].dt.month
    df['pickup_day'] = df['pickup_datetime'].dt.day
    df['pickup_hour'] = df['pickup_datetime'].dt.hour
    df['pickup_minute'] = df['pickup_datetime'].dt.minute
    df['pickup_seconde'] = df['pickup_datetime'].dt.minute * 60
    df['pickup_weekday'] = df['pickup_datetime'].dt.weekday
    return df

In [154]:
df_train = create_datetime(df_train)
df_train = df_train[(df_train['pickup_day'] == df_train['dropoff_datetime'].dt.day)]
df_train = df_train[(df_train['passenger_count'] > 0)]
df_train = df_train.loc[(df_train['pickup_latitude'] < 40.91) & (df_train['pickup_latitude'] > 40.55)]
df_train = df_train.loc[ (df_train['pickup_longitude'] < -73.71) & (df_train['pickup_longitude'] > -74.25)]
df_train = df_train.loc[ (df_train['dropoff_latitude'] < 40.91) & (df_train['dropoff_latitude'] > 40.55)]
df_train = df_train.loc[ (df_train['dropoff_longitude'] < -73.71) & (df_train['dropoff_longitude'] > -74.25)]
df_train.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,pickup_seconde,pickup_weekday
id,,,,,,,,,,,,,,,,,
id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,2016,3,14,17,24,1440,0
id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,2016,6,12,0,43,2580,6
id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,2016,1,19,11,35,2100,1
id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,2016,4,6,19,32,1920,2
id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,2016,3,26,13,30,1800,5


# CONFIG VARIABLES

In [155]:
TARGET = 'trip_duration'
FEATURES = [
    'passenger_count', 'store_and_fwd_flag','pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
    'pickup_year', 'pickup_month', 'pickup_day', 'pickup_hour', 'pickup_minute', 'pickup_seconde', 'pickup_weekday'
]

# CREATE DATASET

In [156]:
def create_dataset(df, features, target):
    map_dict = {'N':0, 'Y':1}
    df['store_and_fwd_flag'] = df['store_and_fwd_flag'].map(map_dict)
    X = df[features]
    y = df[target]
    
    return X, y

In [157]:

X_train, y_train = create_dataset(df_train, FEATURES, TARGET)


X_train.shape, y_train.shape

((1440579, 13), (1440579,))

# Load ShuffleSplit

In [158]:
from sklearn.model_selection import ShuffleSplit

rs = ShuffleSplit(n_splits=4, test_size=.30, train_size=.30)

In [159]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

In [160]:
rf = RandomForestRegressor(n_estimators=12, random_state=42)
losses = -cross_val_score(rf, X_train, y_train, cv=rs, scoring='neg_mean_squared_log_error')
losses = [np.sqrt(l) for l in losses]
np.mean(losses)

0.42398302766051443

In [161]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=12, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [162]:
df_test = pd.read_csv('../input/test.csv', index_col='id', parse_dates=['pickup_datetime'])
df_test = create_datetime(df_test)
df_test.head()

,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,pickup_seconde,pickup_weekday
id,,,,,,,,,,,,,,,
id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N,2016,6,30,23,59,3540,3
id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N,2016,6,30,23,59,3540,3
id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N,2016,6,30,23,59,3540,3
id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N,2016,6,30,23,59,3540,3
id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N,2016,6,30,23,59,3540,3


In [163]:
X_test = df_test[FEATURES]
map_dict = {'N':0, 'Y':1}
X_test['store_and_fwd_flag'] = X_test['store_and_fwd_flag'].map(map_dict)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [164]:
y_pred = rf.predict(X_test)
y_pred.mean()

847.4877675559691

In [165]:
submission = pd.read_csv('../input/sample_submission.csv') 
submission.head()

,id,trip_duration
0,id3004672,959
1,id3505355,959
2,id1217141,959
3,id2150126,959
4,id1598245,959


In [166]:
submission['trip_duration'] = y_pred
submission.head()

,id,trip_duration
0,id3004672,666.083333
1,id3505355,653.000000
2,id1217141,493.666667
3,id2150126,1347.000000
4,id1598245,471.083333


In [167]:
submission.describe()

,trip_duration
count,625134.000000
mean,847.487768
std,603.974426
min,6.583333
25%,440.583333
50%,686.333333
75%,1069.333333
max,28776.916667


In [168]:
submission.to_csv('submission.csv', index=False)

In [169]:
!ls

__notebook_source__.ipynb  submission.csv
